<a href="https://colab.research.google.com/github/SNJ292/snj292.github.io/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

years = list(range(2012, 2022))
url_start = "https://www.basketball-reference.com/draft/NBA_{}.html"

# We're using the url to populate our file 'draft' with drafts between 2012-2022
for year in years:
  url = url_start.format(year)
  data = requests.get(url)

  with open("draft/{}.html".format(year), "w+")as f:
    f.write(data.text)

In [18]:

dfs = []
for year in years:
  #dfs  will hold a list of the dataframes of each draft 
  with open("draft/{}.html".format(year)) as f:
    page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose()
    draft_table = soup.find(id="stats")
    draft = pd.read_html(str(draft_table))[0]
    draft.head()

    dfs.append(draft)
dfs

<class 'pandas.core.frame.DataFrame'>
